# Assignment 3 Group no. 6
### Project members: 
Mar Balibrea Rull, marbr@kth.se
Marcos Fernández Carbonell, marcosfc@kth.se
Tiger Huan Hu Zha, tigerz2@illinois.edu

### Declaration
By submitting this solution, it is hereby declared that all individuals listed above have contributed to the solution, either with code that appear in the final solution below, or with code that has been evaluated and compared to the final solution, but for some reason has been excluded. It is also declared that all project members fully understand all parts of the final solution and can explain it upon request.

It is furthermore declared that the code below is a contribution by the project members only, and specifically that no part of the solution has been copied from any other source (except for lecture slides at the course ID2214) and no part of the solution has been provided by someone not listed as project member above.

It is furthermore declared that it has been understood that no other library/package than the Python 3 standard library, NumPy, pandas and time may be used in the solution for this assignment.

### Instructions
All assignments starting with number 1 below are mandatory. Satisfactory solutions
will give 1 point (in total). If they in addition are good (all parts work more or less 
as they should), completed on time (submitted before the deadline in Canvas) and according
to the instructions, together with satisfactory solutions of assignments starting with 
number 2 below, then the assignment will receive 2 points (in total).

It is highly recommended that you do not develop the code directly within the notebook
but that you copy the comments and test cases to your regular development environment
and only when everything works as expected, that you paste your functions into this
notebook, do a final testing (all cells should succeed) and submit the whole notebook 
(a single file) in Canvas (do not forget to fill in your group number and names above).


## Load NumPy, pandas and time

In [1]:
import numpy as np
import pandas as pd
import time


## Reused functions from Assignment 1

In [4]:
# Copy and paste functions from Assignment 1 here that you need for this assignment

def create_imputation(df):
    
    df_tmp = df.copy()
    
    df_tmp_num = df_tmp.drop(["CLASS","ID"], axis=1, errors="ignore").select_dtypes(include=["int","float"]) # Select only numeric columns
    df_mean = pd.DataFrame(df_tmp_num.mean())
    df_tmp_num[(df_tmp_num.isna().all()).index[df_tmp_num.isna().all()]] = 0 # When all values are missing in a numeric column all values are replaced with by 0

    df_tmp_obj = df_tmp.drop(["CLASS","ID"], axis=1, errors="ignore").select_dtypes(include=["object","category"]) # Select only object and categorical columns
    df_mode = df_tmp_obj.mode().transpose()
    df_tmp_obj[(df_tmp_obj.isna().all()).index[df_tmp_obj.isna().all()]] = "" # When all values are missing in a object or categorical column all values are replaced by ""

    imputation = ((pd.concat([df_mean[0],df_mode[0]], axis=0)).fillna(0)).to_dict() # Create a mapping from column name to new value
    
    df_out = apply_imputation(df, imputation) # Apply imputation
    return df_out, imputation

def apply_imputation(df, imputation):
    
    df_out = df.copy()
    df_out.fillna(value=imputation, inplace=True) # Replace NaN values accoding with the dictionary imputation
    return df_out

def create_bins(df, nobins = 10, bintype = "equal-width"):
    
    # hints 1, 2, 3
    _df = df.select_dtypes(include = ["float", "int"]).drop(columns = ["CLASS", "ID"])
    
    binning = {}
    for col in _df.columns:
        
        if bintype == "equal-width":
            
            res, bins = pd.cut(_df[col], nobins, retbins = True, labels = False, duplicates = "drop")
        elif bintype == "equal-size":
            
            res, bins = pd.qcut(_df[col], nobins, retbins = True, labels = False, duplicates = "drop")
            
        # hint 6
        bins[0] = -np.inf
        bins[-1] = np.inf
        
        binning[col] = tuple(bins)
        
    # hints 4, 5
    df_out = apply_bins(_df, binning)
    return df_out, binning

def apply_bins(df, binning):
    
    # hint 1
    df_out = df.copy()
    
    for col in binning:
        
        # hints 2, 3, 4
        df_out[col] = pd.DataFrame(pd.cut(df_out[col], binning[col], labels = False, duplicates = "drop"))
        df_out[col].astype("category") #.cat.set_categories(np.arange(len(list(binning.values())[0])))
    
    return df_out


## 1. Define the class DecisionTree

In [25]:
# Define the class DecisionTree with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# binning, imputatiom, labels, model
#
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins: no. of bins (default = 10)
# bintype: either "equal-width" (default) or "equal-size"
# min_samples_split: no. of instances required to allow a split (default = 5)
#
# Output from fit:
# nothing
#
# The result of applying this function should be:
#
# self.binning should be a discretization mapping (see Assignment 1) from df
# self.imputation should be an imputation mapping (see Assignment 1) from df
# self.labels should be the categories of the "CLASS" column of df, set to be of type "category" 
# self.model should be a decision tree (for details, see lecture slides), where the leafs return class probabilities
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: First find the available features (excluding "CLASS" and "ID"), then find the class counts, e.g., using 
#         groupby, and calculate the default class probabilities (relative frequencies of the class labels)
# Hint 2: Define a function, e.g., called divide_and_conquer, that takes the above as input together with df 
#         and min_samples_split, and also a nodeno (starting with 0) to keep track of the generated nodes in the tree
# Hint 3: You may represent the tree under construction as a list of nodes (tuples), on the form:
#         (nodeno,"leaf",class_probabilities): corresponding to a leaf node where class_probabilities is a vector
#                                              with the relative class frequencies (ordered according to self.labels)
#         (nodeno,feature,node_dict): corresponding to an internal (non-leaf) node where node_dict is a mapping from
#                                     the possible values of feature to child nodes (their nodenos)
# Hint 4: You may evaluate each feature by a function information_content, which takes the group sizes
#         for each possible value of the feature together with the class counts of each group as input
# Hint 5: The best feature found (with lowest resulting information content) will be used to split the training
#         instances, and each sub-group is used for generating a sub-tree (recursively by divide_and_conquer,
#         see lecture slides for details)
# Hint 6: You may make divide_and_conquer return not only a list of nodes, but also a current_node_no; 
#         by this, each subsequent call to divide_and_conquer for each subset of instances, i.e. for each feature value, 
#         could use current_node_no as a starting point.
#         If you e.g. make the following call:
#
#         current_node_no, node_list = divide_and_conquer(current_node_no, ...)
#
#         then the returned value in current_node_no can be used in the next call to divide_and_conquer.
#         Node that node_list will contain an arbitrary number of tuples, each element corresponding to a node together 
#         with a node number. The first element in the list will have the same number as current_node_no when the call 
#         was made and the last element will have a number one less than current_node_no when returned, e.g., if there is
#         only one (leaf) node in the returned list, then current_node_no will only be incremented by one through the above call.
#         
# Hint 7: The list of nodes output by divide_and_conquer may finally be converted to an array, where each nodeno in the 
#         tuples corresponds to an index of the array
#
# Input to predict:
# self: the object itself
# df: a dataframe
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
#              predictions with estimated class probabilities for each row in df, where the class probabilities
#              are the relative class frequencies in the leaves of the decision tree into which the instances in
#              df fall
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply imputation and binning
# Hint 2: Iterate over the rows calling some sub-function, e.g., make_prediction(nodeno,row), which for a test row
#         finds a leaf node from which class probabilities are obtained
# Hint 3: This sub-function may recursively traverse the tree (represented by an array), starting with the nodeno
#         that corresponds to the root



In [ ]:
class DecisionTree:
    
    def __init__(self, ):
        self.binning = None
        self.imputation = None
        self.labels = None
        self.model = None
        
    def fit(self, df, nobins = 10, bintype = "equal-width", min_samples_split = 5):
        
        # hints 2, 6
        def divide_and_conquer(df, class_counts, min_samples_split, nodeno):
            
            # hints 4, 5
            def information_content():
                
                return feature
                
            tree = []
            _df = df.copy()
            if _df.shape[0] >= min_samples_split: # there's enough instances left
                
                if len(_df["CLASS"].unique()) > 1: # there's more than 1 class left
                    
                    if _df.shape[1] > 0: # there's features left
                        
                        f = information_content()
                        values = _df[f]
                        _df.drop(columns = f)
                        
                        node = nodeno.copy()
                        tree.append((node, f, {}))
                        node++
                        for v in values.unique():
                            
                            # hint 3
                            v_df = _df.copy()
                            v_df = v_df.iloc[v_df[f] == v, :]
                            current_node_no, node_list = divide_and_conquer(v_df, class_counts, min_samples_split, node)
                            
                            tree[nodeno][2].update({v: current_node_no})
                            tree.append(node_list)
                            
                            node = current_node_no + 1
                        
                    else:
                        
                        return nodeno, [(_df["CLASS"].value_counts().argmax())]
                else:
                    
                    return nodeno, [(nodeno, "leaf", _df["CLASS"].unique()[0])]
            else:
                
                # no ho pillo molt
                
            return node, tree
            
            # 1. queden instances? no fas T = label, si continues
            # 2. queden instances amb != classes? no fas T = classe, si continues
            # 3. queden features? no fas T = majority class de I left, si continues
            
            
        _df, self.binning = create_bins(df, nobins, bintype)
        _df, self.imputation = create_imputation(_df)
        self.labels = df["CLASS"].astype("category")
        
        # hint 1
        features = df.copy().columns.drop(["CLASS", "ID"])
        class_counts = {f: {(n[0], n[1]): len(g)
                                for (n, g) in _df.groupby(["CLASS", col])}
                                for f in features.columns.drop(["CLASS", "ID"])} # Mapping from a feature to another mapping, which given a feature value and class label provides the number of training instances with this specific combination
        tree = divide_and_conquer(_df, class_counts, min_samples_split, 0)
        
        # hint 7
        
        
        
#
# Hint 1: First find the available features (excluding "CLASS" and "ID"), then find the class counts, e.g., using 
#         groupby, and calculate the default class probabilities (relative frequencies of the class labels)
# Hint 2: Define a function, e.g., called divide_and_conquer, that takes the above as input together with df 
#         and min_samples_split, and also a nodeno (starting with 0) to keep track of the generated nodes in the tree
# Hint 3: You may represent the tree under construction as a list of nodes (tuples), on the form:
#         (nodeno,"leaf",class_probabilities): corresponding to a leaf node where class_probabilities is a vector
#                                              with the relative class frequencies (ordered according to self.labels)
#         (nodeno,feature,node_dict): corresponding to an internal (non-leaf) node where node_dict is a mapping from
#                                     the possible values of feature to child nodes (their nodenos)
# Hint 4: You may evaluate each feature by a function information_content, which takes the group sizes
#         for each possible value of the feature together with the class counts of each group as input
# Hint 5: The best feature found (with lowest resulting information content) will be used to split the training
#         instances, and each sub-group is used for generating a sub-tree (recursively by divide_and_conquer,
#         see lecture slides for details)
# Hint 6: You may make divide_and_conquer return not only a list of nodes, but also a current_node_no; 
#         by this, each subsequent call to divide_and_conquer for each subset of instances, i.e. for each feature value, 
#         could use current_node_no as a starting point.
#         If you e.g. make the following call:
#
#         current_node_no, node_list = divide_and_conquer(current_node_no, ...)
#
#         then the returned value in current_node_no can be used in the next call to divide_and_conquer.
#         Node that node_list will contain an arbitrary number of tuples, each element corresponding to a node together 
#         with a node number. The first element in the list will have the same number as current_node_no when the call 
#         was made and the last element will have a number one less than current_node_no when returned, e.g., if there is
#         only one (leaf) node in the returned list, then current_node_no will only be incremented by one through the above call.
# Hint 7: The list of nodes output by divide_and_conquer may finally be converted to an array, where each nodeno in the 
#         tuples corresponds to an index of the array


    def predict(self, df):
        
        # hint 3
        def make_prediction(nodeno, row):
            
            node = None # recursively search to look for leaf
            
            return node
        
        # hint 1
        _df = df.columns.drop(["CLASS", "ID"])
        _df = apply_bins(df, self.binning)
        _df = apply_imputation(_d, self.imputation)
        
        # hint 2
        for row in _df.index:
            
            _df.iloc[row, :] = make_prediction(0, row) # nodeno = 0????
            
        
        # predictions: a dataframe with class labels as column names and the rows corresponding to
#              predictions with estimated class probabilities for each row in df, where the class probabilities
#              are the relative class frequencies in the leaves of the decision tree into which the instances in
#              df fall
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply imputation and binning
# Hint 2: Iterate over the rows calling some sub-function, e.g., make_prediction(nodeno,row), which for a test row
#         finds a leaf node from which class probabilities are obtained
# Hint 3: This sub-function may recursively traverse the tree (represented by an array), starting with the nodeno
#         that corresponds to the root
        

In [14]:
# Test your code (leave this part unchanged, except for if auc is undefined)

glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

tree_model = DecisionTree()

test_labels = glass_test_df["CLASS"]

nobins_values = [5,10]
bintype_values = ["equal-width","equal-size"]
min_samples_split_values = [3,5,10]
parameters = [(nobins,bintype,min_samples_split) for nobins in nobins_values for bintype in bintype_values 
              for min_samples_split in min_samples_split_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    tree_model.fit(glass_train_df,nobins=parameters[i][0],bintype=parameters[i][1],min_samples_split=parameters[i][2])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = tree_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([nobins_values,bintype_values,min_samples_split_values]),
                       columns=["Accuracy","Brier score","AUC"])

results



Training time (5, 'equal-width', 3): 1.58 s.
Testing time (5, 'equal-width', 3): 0.11 s.
Training time (5, 'equal-width', 5): 1.33 s.
Testing time (5, 'equal-width', 5): 0.11 s.
Training time (5, 'equal-width', 10): 0.74 s.
Testing time (5, 'equal-width', 10): 0.11 s.
Training time (5, 'equal-size', 3): 1.60 s.
Testing time (5, 'equal-size', 3): 0.07 s.
Training time (5, 'equal-size', 5): 0.84 s.
Testing time (5, 'equal-size', 5): 0.07 s.
Training time (5, 'equal-size', 10): 0.44 s.
Testing time (5, 'equal-size', 10): 0.07 s.
Training time (10, 'equal-width', 3): 2.47 s.
Testing time (10, 'equal-width', 3): 0.06 s.
Training time (10, 'equal-width', 5): 1.56 s.
Testing time (10, 'equal-width', 5): 0.06 s.
Training time (10, 'equal-width', 10): 0.88 s.
Testing time (10, 'equal-width', 10): 0.06 s.
Training time (10, 'equal-size', 3): 1.51 s.
Testing time (10, 'equal-size', 3): 0.07 s.
Training time (10, 'equal-size', 5): 1.32 s.
Testing time (10, 'equal-size', 5): 0.07 s.
Training time (

Accuracy  Brier score       AUC
5  equal-width 3   0.635514     0.566861  0.794838
               5   0.644860     0.530257  0.812039
               10  0.579439     0.540913  0.802684
   equal-size  3   0.644860     0.678333  0.757026
               5   0.635514     0.644065  0.759834
               10  0.598131     0.652672  0.743455
10 equal-width 3   0.654206     0.566439  0.816216
               5   0.626168     0.529316  0.832679
               10  0.598131     0.503810  0.845034
   equal-size  3   0.504673     0.905079  0.656123
               5   0.504673     0.902743  0.655598
               10  0.532710     0.868821  0.669287

In [15]:
train_labels = glass_train_df["CLASS"]
tree_model.fit(glass_train_df,min_samples_split=1)
predictions = tree_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.97
AUC on training set: 1.00
Brier score on training set: 0.03


### Comment on assumptions, things that do not work properly, etc.


## 2. Define the class DecisionForest

In [9]:
# Define the class DecisionForest with three functions __init__, fit and predict (after the comments):
#
# Input to __init__: 
# self: the object itself
#
# Output from __init__:
# nothing
# 
# This function does not return anything but just initializes the following attributes of the object (self) to None:
# binning, imputatiom, labels, model
#
# Input to fit:
# self: the object itself
# df: a dataframe (where the column names "CLASS" and "ID" have special meaning)
# nobins: no. of bins (default = 10)
# bintype: either "equal-width" (default) or "equal-size"
# min_samples_split: no. of instances required to allow a split (default = 5)
# random_features: no. of features to evaluate at each split (default = 2), 0 means all features (no random sampling)
# notrees: no. of trees in the forest (default = 10)
#
# Output from fit:
# nothing
#
# The result of applying this function should be:
#
# self.binning should be a discretization mapping (see Assignment 1) from df
# self.imputation should be an imputation mapping (see Assignment 1) from df
# self.labels should be the categories of the "CLASS" column of df, set to be of type "category" 
# self.model should be a random forest (for details, see lecture slides)
# Note that the function does not return anything but just assigns values to the attributes of the object.
#
# Hint 1: Redefine divide_and_conquer to take one additional argument; random_features, and instead of
#         evaluating all features choose a random subset, e.g., by np.random.choice (without replacement)
# Hint 2: Generate each tree in the forest from a bootstrap replicate of df, e.g., by np.random.choice 
#         (with replacement) from the index values of df.
#
# Input to predict:
# self: the object itself
# df: a dataframe
# 
# Output from predict:
# predictions: a dataframe with class labels as column names and the rows corresponding to
#              predictions with estimated class probabilities for each row in df, where the class probabilities
#              are the mean of all relative class frequencies in the leaves of the forest into which the instances in
#              df fall
#
# Hint 1: Drop any "CLASS" and "ID" columns first and then apply imputation and binning
# Hint 2: Iterate over the rows calling some sub-function, e.g., make_prediction(row), which for a test row
#         finds all leaf nodes and calculates the average of their class probabilities



In [18]:
glass_train_df = pd.read_csv("glass_train.txt")

glass_test_df = pd.read_csv("glass_test.txt")

forest_model = DecisionForest()

test_labels = glass_test_df["CLASS"]

min_samples_split_values = [1,2,5]
random_features_values = [1,2,5]

parameters = [(min_samples_split,random_features) for min_samples_split in min_samples_split_values 
              for random_features in random_features_values]

results = np.empty((len(parameters),3))

for i in range(len(parameters)):
    t0 = time.perf_counter()
    forest_model.fit(glass_train_df,min_samples_split=parameters[i][0],random_features=parameters[i][1])
    print("Training time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    t0 = time.perf_counter()
    predictions = forest_model.predict(glass_test_df)
    print("Testing time {0}: {1:.2f} s.".format(parameters[i],time.perf_counter()-t0))
    results[i] = [accuracy(predictions,test_labels),brier_score(predictions,test_labels),
                  auc(predictions,test_labels)] # Assuming that you have defined auc - remove otherwise

results = pd.DataFrame(results,index=pd.MultiIndex.from_product([min_samples_split_values,random_features_values]),
                       columns=["Accuracy","Brier score","AUC"])

results

Training time (1, 1): 7.51 s.
Testing time (1, 1): 0.09 s.
Training time (1, 2): 10.65 s.
Testing time (1, 2): 0.09 s.
Training time (1, 5): 19.78 s.
Testing time (1, 5): 0.09 s.
Training time (2, 1): 7.97 s.
Testing time (2, 1): 0.09 s.
Training time (2, 2): 12.01 s.
Testing time (2, 2): 0.10 s.
Training time (2, 5): 19.42 s.
Testing time (2, 5): 0.10 s.
Training time (5, 1): 4.57 s.
Testing time (5, 1): 0.10 s.
Training time (5, 2): 7.31 s.
Testing time (5, 2): 0.09 s.
Training time (5, 5): 9.75 s.
Testing time (5, 5): 0.09 s.


Accuracy  Brier score       AUC
1 1  0.644860     0.477912  0.860736
  2  0.710280     0.411981  0.892842
  5  0.710280     0.421511  0.895716
2 1  0.663551     0.452421  0.872447
  2  0.626168     0.411684  0.911137
  5  0.654206     0.435847  0.881621
5 1  0.644860     0.457859  0.865636
  2  0.719626     0.410616  0.903016
  5  0.616822     0.455782  0.883897

In [24]:
train_labels = glass_train_df["CLASS"]
forest_model.fit(glass_train_df,min_samples_split=1)
predictions = forest_model.predict(glass_train_df)
print("Accuracy on training set: {0:.2f}".format(accuracy(predictions,train_labels)))
print("AUC on training set: {0:.2f}".format(auc(predictions,train_labels)))
print("Brier score on training set: {0:.2f}".format(brier_score(predictions,train_labels)))

Accuracy on training set: 0.96
AUC on training set: 1.00
Brier score on training set: 0.12


### Comment on assumptions, things that do not work properly, etc.